In [1]:
import smtplib
import ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import base64
import papermill as pm
import os 
import sys
import re 
import datetime
import pickle
import nbconvert
import nbformat
from traitlets.config import Config
from nbconvert import HTMLExporter
from nbconvert.writers import FilesWriter

In [2]:
!pwd

/home/malcolm/petfinder


## Run Notebooks

In [4]:
today_str = str(datetime.datetime.now().date())

In [5]:
notebook = "S1_Petfinder_prod.ipynb"
new_notebook = f"run_notebooks/S1_Petfinder_prod_{today_str}.ipynb"

In [6]:
try:
    pm.execute_notebook(notebook
              , new_notebook)
    error = None
except Exception as e:
    error = str(e)
    print("Error")
    print(error)


Error

---------------------------------------------------------------------------
Exception encountered at "In [16]":
---------------------------------------------------------------------------
WebDriverException                        Traceback (most recent call last)
<ipython-input-16-a1f41e2291d8> in <module>
      1 print(sys.platform)
      2 if sys.platform == 'linux':
----> 3     browser = webdriver.Chrome('/usr/bin/chromedriver', options=options)
      4 elif sys.platform =='darwin':
      5     browser = webdriver.Chrome('/Users/malcolmtaylor/node_modules/chromedriver/lib/chromedriver/chromedriver'

~/venv/lib/python3.7/site-packages/selenium/webdriver/chrome/webdriver.py in __init__(self, executable_path, port, options, service_args, desired_capabilities, service_log_path, chrome_options, keep_alive)
     79                     remote_server_addr=self.service.service_url,
     80                     keep_alive=keep_alive),
---> 81                 desired_capabilities=desire

## Email

In [12]:
def wrap_as_html(new_notebook_name):
    
    with open(new_notebook_name, 'r') as hnd:
        notebook_txt = hnd.read()
    nb_notebook = nbformat.reads(notebook_txt, as_version=4)
    html_exporter = HTMLExporter()
    html_exporter.template_file = 'basic'
    (body, resources) = html_exporter.from_notebook_node(nb_notebook)

    writer1 = FilesWriter()
    html_name = re.sub('.ipynb', '', new_notebook_name)
    writer1.write(output = body, resources = resources, notebook_name=html_name)
    html_name = html_name + '.html'


    return(html_name)

In [13]:
def create_html_attachment(html_name):
    
    with open(html_path, 'r') as hnd:
        html = hnd.read()
    part1 = MIMEText(html, 'html')
    part1.add_header("Content-Disposition", f"attachment; filename= {html_path}")
    
    return(part1)

In [14]:
gmail_login = "malctaylordev@gmail.com"
sender_email = "malctaylordev@gmail.com"
reciever_email = "malctaylor15@gmail.com"
# reciever_email = gmail_login

In [15]:
with open('/home/malcolm/credentials/email_pw.pkl', 'rb') as hnd:
    pw = pickle.load(hnd)['pw']

In [16]:
message = MIMEMultipart()
message["From"] = sender_email
message["To"] = reciever_email
port = 465
context = ssl.create_default_context()

In [18]:
if isinstance(error, str):
    print("Error workflow...")
    subject = "Error in Petfinder: "+today_str
    message["Subject"] = subject

    
    body = """
        There was an error in Petfinder Data Collection. 

        Stack Trace: 

        """ + error
    
    text = MIMEText(body, "plain")
    
    message.attach(text)

    try:
        html_path = wrap_as_html(new_notebook)
        html_attachment = create_html_attachment(html_path)
        message.attach(html_attachment)

    except Exception as e: 
        print(e)
        body = body + " \n\n Error attaching notebook... WIP \n\n" + str(e)
  
    
    with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
        server.login(gmail_login, pw)
        server.sendmail(gmail_login, reciever_email, message.as_string())
else: 
    os.system("rm "+ new_notebook)
    print("removed notebook: ", new_notebook)

removed notebook:  run_notebooks/get_latest_songs_prod_2020-08-02.ipynb
